In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker
from detector import Detector
from teams import TeamAffiliation
from keypoint_detector import KeypointDetector
from mplsoccer import Pitch
import matplotlib.pyplot as plt
import utils
from player_position import PlayerPosition
from radar import Radar
from video_generator import VideoGenerator

In [ ]:
SOURCE_PATH = r""
DEST_PATH = r""

In [ ]:
model = YOLO("training/runs/detect/train4/weights/best.pt")
model_keypoints = YOLO("training/runs/pose/train8/weights/best.pt")
detector = Detector(model, 0.5)
keypoint_detector = KeypointDetector(model_keypoints)
tracker = Tracker(detector)
positions = PlayerPosition()

In [ ]:
# images_path = r'data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1'
# frames = utils.load_frames_from_images(images_path)
frames = utils.load_frames_from_video(SOURCE_PATH)

In [ ]:
tracks = tracker.track(frames)

In [ ]:
keypoints = keypoint_detector.detect(frames)

In [ ]:
keypoints_list = keypoint_detector.fill_blank(frames)

In [ ]:
player_positions = positions.get_players_position(keypoints_list, tracks)

In [ ]:
teams = TeamAffiliation(tracks, frames)

In [ ]:
player_teams: list[np.array] = teams.get_teams()

In [ ]:
position_df = utils.parse_player_data(player_positions, tracks, player_teams)

In [ ]:
video = teams.annotate_frames()

In [ ]:
radar = Radar(position_df, teams.get_team_colors())

In [ ]:
video_generator = VideoGenerator(video, radar)

In [ ]:
video_generator.generate_video_with_radar(DEST_PATH, n_frames_back=45)